In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Student data

In [25]:
student_data_df = pd.read_sql('select * from student', conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [27]:
student_data_df.rename(columns={
    'ID_Student': 'Student_ID__c',
    'StudentID': 'Student_No__c',
    'LastName': 'LastName__c',
    'FirstName': 'FirstName__c',
    'MiddleName': 'MiddleName__c',
    'BirthDate': 'BirthDate__c',
    'Gender': 'Gender__c'
}, inplace=True)

student_data_df.head()

,Student_ID__c,Student_No__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [28]:
# select columns
#student_data_df = student_data_df[['ID_Student__c','LastName__c','FirstName__c', 'MiddleName__c', 'BirthDate__c', 'Gender__c']]
#student_data_df.head()

In [29]:
# Create record - list of dictionaries
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'Student_ID__c': 33,
  'Student_No__c': '25004961',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': 34,
  'Student_No__c': '25003514',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Lana',
  'MiddleName__c': 'Cecille',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': 35,
  'Student_No__c': '25005833',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': 36,
  'Student_No__c': '25002589',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Kimberly',
  'MiddleName__c': 'Genevieve',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': 37,
  'Student_No__c': '25007185',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': 38,
  'Student_No__c': '25006014',
  'LastNam

In [30]:
# Migrate data to Salesforce
for rec in student_data_records:
    record = {
        'Student_ID__c': rec['Student_ID__c'],
        'Student_No__c': rec['Student_No__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': rec['BirthDate__c'],
        'Gender__c': rec['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

## Create Student Lookup Table

In [31]:
student_lookup_list = []

data = sf.query_all_iter("select Student_ID__c, Name from Student__c")
for row in data:
    rec = {
        "ID_Student__c": row['Name'],
        "Student_ID__c": row['Student_ID__c']
    }
    student_lookup_list.append(rec)

student_lookup_list

[{'ID_Student__c': 'a034x000002idey', 'Student_ID__c': '37'},
 {'ID_Student__c': 'a034x000002idfw', 'Student_ID__c': '49'},
 {'ID_Student__c': 'a034x000002idgB', 'Student_ID__c': '52'},
 {'ID_Student__c': 'a034x000002idfh', 'Student_ID__c': '46'},
 {'ID_Student__c': 'a034x000002idga', 'Student_ID__c': '57'},
 {'ID_Student__c': 'a034x000002idee', 'Student_ID__c': '33'},
 {'ID_Student__c': 'a034x000002idg1', 'Student_ID__c': '50'},
 {'ID_Student__c': 'a034x000002idfr', 'Student_ID__c': '48'},
 {'ID_Student__c': 'a034x000002idgp', 'Student_ID__c': '60'},
 {'ID_Student__c': 'a034x000002idfN', 'Student_ID__c': '42'},
 {'ID_Student__c': 'a034x000002idf8', 'Student_ID__c': '39'},
 {'ID_Student__c': 'a034x000002idgf', 'Student_ID__c': '58'},
 {'ID_Student__c': 'a034x000002idej', 'Student_ID__c': '34'},
 {'ID_Student__c': 'a034x000002idf3', 'Student_ID__c': '38'},
 {'ID_Student__c': 'a034x000002idfI', 'Student_ID__c': '41'},
 {'ID_Student__c': 'a034x000002idgL', 'Student_ID__c': '54'},
 {'ID_St

In [32]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df.head()

,ID_Student__c,Student_ID__c
0,a034x000002idey,37
1,a034x000002idfw,49
2,a034x000002idgB,52
3,a034x000002idfh,46
4,a034x000002idga,57


### Create Class lookup table

In [35]:
class_lookup_list = []

data = sf.query_all_iter('select ID_Class__c, Name from Class__c')
for row in data:
    rec = {
        "Class_ID__c": row['Name'],
        "ID_Class__c": row['ID_Class__c']
    }
    
    class_lookup_list.append(rec)

class_lookup_list

[{'Class_ID__c': 'a014x000008Tj9e', 'ID_Class__c': '2'},
 {'Class_ID__c': 'a014x000008Tj9Z', 'ID_Class__c': '1'},
 {'Class_ID__c': 'a014x000008Tj9j', 'ID_Class__c': '3'},
 {'Class_ID__c': 'a014x000008Tj9o', 'ID_Class__c': '4'}]

In [36]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df.head()

,Class_ID__c,ID_Class__c
0,a014x000008Tj9e,2
1,a014x000008Tj9Z,1
2,a014x000008Tj9j,3
3,a014x000008Tj9o,4


In [44]:
class_lookup_df['ID_Class__c'] = class_lookup_df['ID_Class__c'].astype(str)
class_lookup_df.head()

,Class_ID__c,ID_Class__c
0,a014x000008Tj9e,2
1,a014x000008Tj9Z,1
2,a014x000008Tj9j,3
3,a014x000008Tj9o,4


#### Query the Classparticipant table from MySQL

In [38]:
query = '''
   select 
      *
   from classparticipant

'''
classparticipant_data_df = pd.read_sql(query, conn)
classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-09-16,None
1,2,34,1,2020-09-16,None
2,3,35,1,2020-09-16,None
3,4,62,1,2020-09-16,None
4,5,36,1,2020-09-16,None


In [39]:
classparticipant_data_df.rename(columns={
    'ID_ClassParticipant':'ID_ClassParticipant__c',
    'ID_Student':'Student_ID__c',
    'ID_Class':'ID_Class__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c'
}, inplace=True)

classparticipant_data_df.head()

,ID_ClassParticipant__c,Student_ID__c,ID_Class__c,StartDate__c,EndDate__c
0,1,33,1,2020-09-16,None
1,2,34,1,2020-09-16,None
2,3,35,1,2020-09-16,None
3,4,62,1,2020-09-16,None
4,5,36,1,2020-09-16,None


In [40]:
#classparticipant_data_df = classparticipant_data_df[['ID_Student__c', 'ID_ClassParticipant__c', 'ID_Class__c', 'StartDate__c', 'EndDate__c']]
#classparticipant_data_df.head()

In [41]:
classparticipant_data_df['Student_ID__c'] = classparticipant_data_df['Student_ID__c'].astype(str)
classparticipant_data_df.head()

,ID_ClassParticipant__c,Student_ID__c,ID_Class__c,StartDate__c,EndDate__c
0,1,33,1,2020-09-16,None
1,2,34,1,2020-09-16,None
2,3,35,1,2020-09-16,None
3,4,62,1,2020-09-16,None
4,5,36,1,2020-09-16,None


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [14]:
student_lookup_df.head()

,ID_Student__c,Student_ID__c
0,a034x000002idYG,48
1,a034x000002idYz,57
2,a034x000002idYQ,50
3,a034x000002idXh,41
4,a034x000002idXI,36


In [47]:
class_lookup_df.head()

,Class_ID__c,ID_Class__c
0,a014x000008Tj9e,2
1,a014x000008Tj9Z,1
2,a014x000008Tj9j,3
3,a014x000008Tj9o,4


In [49]:
classparticipant_data_df.head()

,ID_ClassParticipant__c,Student_ID__c,ID_Class__c,StartDate__c,EndDate__c
0,1,33,1,2020-09-16,None
1,2,34,1,2020-09-16,None
2,3,35,1,2020-09-16,None
3,4,62,1,2020-09-16,None
4,5,36,1,2020-09-16,None


In [45]:
classparticipant_data_merge1 = pd.merge(classparticipant_data_df, student_lookup_df, how='left')
classparticipant_data_merge1.head()

,ID_ClassParticipant__c,Student_ID__c,ID_Class__c,StartDate__c,EndDate__c,ID_Student__c
0,1,33,1,2020-09-16,None,a034x000002idee
1,2,34,1,2020-09-16,None,a034x000002idej
2,3,35,1,2020-09-16,None,a034x000002ideo
3,4,62,1,2020-09-16,None,a034x000002idgz
4,5,36,1,2020-09-16,None,a034x000002idet


In [53]:
#class_lookup_list['ID_Class__c'] = class_lookup_list['ID_Class__c'].astype(str)
#class_lookup_list.head()

In [54]:
classparticipant_data_merge1['ID_Class__c'] = classparticipant_data_merge1['ID_Class__c'].astype(str)
classparticipant_data_merge1.head()

,ID_ClassParticipant__c,Student_ID__c,ID_Class__c,StartDate__c,EndDate__c,ID_Student__c
0,1,33,1,2020-09-16,None,a034x000002idee
1,2,34,1,2020-09-16,None,a034x000002idej
2,3,35,1,2020-09-16,None,a034x000002ideo
3,4,62,1,2020-09-16,None,a034x000002idgz
4,5,36,1,2020-09-16,None,a034x000002idet


In [55]:
classparticipant_data_merge2 = pd.merge(classparticipant_data_merge1, class_lookup_df, how='left')
classparticipant_data_merge2.head()

,ID_ClassParticipant__c,Student_ID__c,ID_Class__c,StartDate__c,EndDate__c,ID_Student__c,Class_ID__c
0,1,33,1,2020-09-16,None,a034x000002idee,a014x000008Tj9Z
1,2,34,1,2020-09-16,None,a034x000002idej,a014x000008Tj9Z
2,3,35,1,2020-09-16,None,a034x000002ideo,a014x000008Tj9Z
3,4,62,1,2020-09-16,None,a034x000002idgz,a014x000008Tj9Z
4,5,36,1,2020-09-16,None,a034x000002idet,a014x000008Tj9Z


In [56]:
classparticipant_data_merge2['StartDate__c'] = pd.to_datetime(classparticipant_data_merge2['StartDate__c']).dt.date
classparticipant_data_merge2.head()

,ID_ClassParticipant__c,Student_ID__c,ID_Class__c,StartDate__c,EndDate__c,ID_Student__c,Class_ID__c
0,1,33,1,2020-09-16,None,a034x000002idee,a014x000008Tj9Z
1,2,34,1,2020-09-16,None,a034x000002idej,a014x000008Tj9Z
2,3,35,1,2020-09-16,None,a034x000002ideo,a014x000008Tj9Z
3,4,62,1,2020-09-16,None,a034x000002idgz,a014x000008Tj9Z
4,5,36,1,2020-09-16,None,a034x000002idet,a014x000008Tj9Z


In [64]:
# Create record
classparticipant_data_records = classparticipant_data_merge2.to_dict(orient='records')
classparticipant_data_records

[{'ID_ClassParticipant__c': 1,
  'Student_ID__c': '33',
  'ID_Class__c': '1',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Student__c': 'a034x000002idee',
  'Class_ID__c': 'a014x000008Tj9Z'},
 {'ID_ClassParticipant__c': 2,
  'Student_ID__c': '34',
  'ID_Class__c': '1',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Student__c': 'a034x000002idej',
  'Class_ID__c': 'a014x000008Tj9Z'},
 {'ID_ClassParticipant__c': 3,
  'Student_ID__c': '35',
  'ID_Class__c': '1',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Student__c': 'a034x000002ideo',
  'Class_ID__c': 'a014x000008Tj9Z'},
 {'ID_ClassParticipant__c': 4,
  'Student_ID__c': '62',
  'ID_Class__c': '1',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Student__c': 'a034x000002idgz',
  'Class_ID__c': 'a014x000008Tj9Z'},
 {'ID_ClassParticipant__c': 5,
  'Student_ID__c': '36',
  'ID_Class__c': '1',
  'StartDate__c': datetime.date(2020, 

## Insert `Class` Records into Salesforce

In [65]:
for rec in classparticipant_data_records:
    
    record = {
        'ID_ClassParticipant__c': rec['ID_ClassParticipant__c'],
        'Student_ID__c': rec['Student_ID__c'],
        'ID_Class__c': rec['ID_Class__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'EndDate__c': str(rec['EndDate__c']),
        'ID_Student__c': rec['ID_Student__c'],
        'Class_ID__c': rec['Class_ID__c']
    }
    
    try:
        sf.Classparticipant__C.create(record)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [61]:
classparticipant_records = sf.query("SELECT Id FROM Classparticipant__c")
recs_to_delete = [{'Id': r['Id']} for r in classparticipant_records['records']]
recs_to_delete

[{'Id': 'a024x000002gVnlAAE'},
 {'Id': 'a024x000002gVnpAAE'},
 {'Id': 'a024x000002gVnuAAE'},
 {'Id': 'a024x000002gVnzAAE'},
 {'Id': 'a024x000002gVo4AAE'},
 {'Id': 'a024x000002gVo9AAE'},
 {'Id': 'a024x000002gVoEAAU'},
 {'Id': 'a024x000002gVoJAAU'},
 {'Id': 'a024x000002gVoOAAU'},
 {'Id': 'a024x000002gVoTAAU'},
 {'Id': 'a024x000002gVoYAAU'},
 {'Id': 'a024x000002gVodAAE'},
 {'Id': 'a024x000002gVoiAAE'},
 {'Id': 'a024x000002gVonAAE'},
 {'Id': 'a024x000002gVosAAE'},
 {'Id': 'a024x000002gVoxAAE'},
 {'Id': 'a024x000002gVp2AAE'},
 {'Id': 'a024x000002gVp7AAE'},
 {'Id': 'a024x000002gVpCAAU'},
 {'Id': 'a024x000002gVpHAAU'},
 {'Id': 'a024x000002gVpMAAU'},
 {'Id': 'a024x000002gVpRAAU'},
 {'Id': 'a024x000002gVpWAAU'},
 {'Id': 'a024x000002gVpbAAE'},
 {'Id': 'a024x000002gVpgAAE'},
 {'Id': 'a024x000002gVplAAE'},
 {'Id': 'a024x000002gVpqAAE'},
 {'Id': 'a024x000002gVpvAAE'}]

In [62]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [63]:
for rec in recs_to_delete:
    try:
        sf.classparticipant__c.delete(rec['Id'])
    except Exception as e:
        print(e)